In [1]:
import cobra
from cobra.io import load_json_model, save_json_model
from cobra import Model, Reaction, Metabolite
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
ipsc = load_json_model('../data/gems/iPSC_gf.json')

In [3]:
ipsc.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
arg__L_e,EX_arg__L_e,10,6,8.70%
asn__L_e,EX_asn__L_e,2.732,4,1.58%
asp__L_e,EX_asp__L_e,10,4,5.80%
chol_e,EX_chol_e,1.982,5,1.44%
cys__L_e,EX_cys__L_e,1.087,3,0.47%
glc__D_e,EX_glc__D_e,10,6,8.70%
gln__L_e,EX_gln__L_e,10,5,7.25%
h2o_e,EX_h2o_e,10,0,0.00%
h_e,EX_h_e,10,0,0.00%
his__L_e,EX_his__L_e,10,6,8.70%


In [ ]:
##########